In [12]:
import requests
import json
import csv
import time, datetime,os
import random
from bs4 import BeautifulSoup as bs

In [ ]:
dt = datetime.datetime.now()
dt.year
dt.month

In [14]:
now = datetime.datetime.now()
year_list = range (2004,now.year+1) #since 2007 to this year
month_list = range(1,13)  # 12 months
day_list = range(1,32) # 31 days

In [ ]:
for year in year_list:
    for month in month_list:
        for day in day_list:
            if (dt.year == year and month > dt.month) :break  # break loop while month over current month
            if ((month == 4 or month == 6 or month == 9 or month == 11) and day == 31 ) :continue # these 4 month only have 30 days
            if (year == 2004 or year == 2008 or year == 2012 or year == 2016) :
                if(month == 2 and day > 29) : continue # leap year --> there are 29 days in February
            elif (month == 2 and day > 28) : continue # there are only 28 days in February
            
            yy  = str(year)
            mm  = month
            dd  = day
            # E:\KEN\課程\大四下\資產管理策略與應用\ETF追蹤\ETF_history
            directory = 'E:/KEN/課程/大四下/資產管理策略與應用/ETF追蹤/ETF_history'+'/' + str(yy) +'/'       #setting directory
            filename = str(yy)+str("%02d"%mm)+str("%02d"%dd)+'.csv'          #setting file name
            
            smt = get_webmsg(year ,month ,day)           #put the data into smt 
            makedirs (year, month)                  #create directory function
            write_csv (directory, filename, smt)    # write files into CSV
            
            sleep_time = random_time()
            time.sleep(sleep_time)

In [15]:
def get_webmsg (year, month , day):
    date = str (year) + "{0:0=2d}".format(month) +"{0:0=2d}".format(day)  ## format is yyyymmdd
    url_twse = 'http://www.twse.com.tw/exchangeReport/MI_INDEX?response=json&date='+date+'&type=0099P'
    res =requests.post(url_twse,)
    
    # print(url_twse)
    # print(res)
    print(date)
    
    soup = bs(res.text , 'html.parser')
    smt = json.loads(soup.text)     #convert data into json
    return smt

In [16]:
def write_csv(directory,filename,smt) :
    if(smt['stat'] == '查詢日期小於93年2月11日，請重新查詢!' or smt['stat'] == '很抱歉，沒有符合條件的資料!'):
        return
    writefile = directory + filename               #set output file name
    outputFile = open(writefile,'w',newline='')
    outputWriter = csv.writer(outputFile)
    head = ''.join(smt['title'].split())
    a = [head,""]
    outputWriter.writerow(a)
    outputWriter.writerow(smt['fields1'])
    for data in (smt['data1']):
        if('-' in data[9]):
            data[9] = '-'
        if('+' in data[9]):
            data[9] = '+'
        outputWriter.writerow(data)

    outputFile.close()

In [17]:
#create a directory in the current one doesn't exist
def makedirs (year, month):
    yy      = str(year)
    mm       = str(month)
    directory =  'E:/KEN/課程/大四下/資產管理策略與應用/ETF追蹤/ETF_history'+'/' + yy +'/'
    if not os.path.isdir(directory):
        os.makedirs (directory)  # os.makedirs able to create multi folders

In [20]:
# random time ( 10 - 20 )
def random_time():
    return (math.floor(random.random()*10) + 11)